In [1]:
!pip install ninja -q

In [2]:
import torch
import dnnlib
import numpy as np
import matplotlib.pyplot as plt
import legacy
import ninja

device = torch.device('cuda')
pkl_path = '/notebooks/training-runs/00006-faces80k256-cond-paper256-batch256-ada-target0.5-resumecustom/network-snapshot-001433.pkl'

In [3]:
with dnnlib.util.open_url(pkl_path) as f:
    G = legacy.load_network_pkl(f)['G'].to(device)# torch.nn.Module

In [ ]:
plt.figure(figsize=(15,24)) # specifying the overall grid size

for j in range(8):
    c = torch.zeros([1, G.c_dim], device = device)
    c[:, j] = 1
    
    for i in range(5):
        z = torch.randn([1, G.z_dim], device = device) 
        img = G(z, c)
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        plt.subplot(8,5,5*j+i+1) 
        plt.imshow(img[0].cpu().numpy())

plt.show()

In [5]:
with dnnlib.util.open_url(pkl_path) as f:
    G = legacy.load_network_pkl(f)['G'].to(device)# torch.nn.Module
with dnnlib.util.open_url(pkl_path) as f:
    D = legacy.load_network_pkl(f)['D'].to(device)# torch.nn.Module

In [146]:
d_array = np.array([0.0])

for j in range(100):
    c = torch.zeros([1, G.c_dim], device = device)
    c[:, 1] = 1
    
    z = torch.randn([1, G.z_dim], device = device) 
    img = G(z, c)
    
    c = torch.zeros([1, G.c_dim], device = device)
    c[:, 5] = 1
    d_array = np.append(d_array, D(img, c).cpu().numpy()[0])

In [147]:
d_array = d_array[1:]
print(d_array)

[-0.6720438  -0.62957281 -0.95796245 -0.87510699 -0.43537685 -0.63005978
 -0.95415574 -0.73160899  0.33689258  0.04281749 -0.15546861 -0.57104427
 -0.46038973 -0.50884974 -0.68602306 -0.59358847 -0.72240102  0.02726357
  0.33330679 -0.09254584 -0.2541362   0.42230842 -0.23233593 -0.67098582
 -0.42114994 -0.54224497 -1.2460351  -0.86778867 -0.53375322 -0.33080691
 -0.02260392 -0.14189111  0.49695307 -0.37513581 -0.57995582 -0.3023544
 -0.42118844 -0.51490748 -0.3105551  -0.77798212 -0.22703177 -0.67818958
 -1.03003895 -0.53340089 -0.78642923 -1.39160419 -0.68348157  0.4372862
 -0.82216084  0.06487411 -1.3597101  -0.64454371 -0.06306008 -0.91133612
 -0.98157042  0.24692978 -0.80173063 -0.0483285  -0.51209569 -1.30141628
 -0.97448283 -0.29308128  0.05263532 -0.53658551 -0.68242341 -0.02674106
 -0.68959695 -0.17025213 -0.52011299 -1.15011549 -0.41814452 -0.34191573
 -0.73772955 -0.11165205 -0.98279184 -0.95395589 -0.60372567 -0.39808583
 -0.48843119 -1.10297215 -0.68666089 -0.40340137 -0.0

In [148]:
def getRandomSamplesInNSphere(N , R , numberOfSamples):
    # Return 'numberOfSamples' samples of vectors of dimension N 
    # with an uniform distribution inside the N-Sphere of radius R.
    # RATIONALE: https://math.stackexchange.com/q/87238
    
    randomnessGenerator = np.random.default_rng()
    
    X = randomnessGenerator.normal(size=(numberOfSamples , N))
    U = randomnessGenerator.random((numberOfSamples , 1)) 
    
    return R * U**(1/N) / np.sqrt(np.sum(X**2, 1, keepdims=True)) * X

In [5]:
import numpy as np
l = []

for i in range(5):
    z = np.random.RandomState(i).randn(1, 10) 
    l.append(z[0])

z_array = np.asarray(l)
z_array.shape

(5, 10)